# <center>GoJournal: a verified, concurrent, crash-safe journaling system, or</center>
# <center>Panic! at the Disk-Go</center>


## Go-Journal

### Journaling

* #### Journaling enables recovery from crashes in reliable storage systems using a log
* #### Crashes may occur at any time during system execution, including during recovery
* #### Journaling is particularly difficult to test due to concurrency, and recovery code runs only rarely
* #### The journal is used here to implement a NFS storage server

<img src="api.png" width="50%" style="margin: 0" />


### Verification and Proof

* #### Conventional testing (unit testing, integration testing)
* #### Property-based testing (QuickCheck, ...)
* #### Finite model checking (TLA+, ...)
* #### Mechanized proof (Coq, Isabelle, Agda, Lean, ...)


### Go-Journal Verification Stack

* #### Goose: translation of Go programs to Coq
* #### Perennial: high-level properties/proofs for concurrency and crash safety
* #### Iris: logic for proving concurrent imperative programs
* #### Coq: core functional language and interactive theorem prover

<hr/>

Key here is that the system can be implemented in a conventional programming language and verified in a proof language

### Proof Structure

<img src="layers.png" width="25%" style="margin: 0" />

* #### Modularity over Go-Journal layers/subsystems
* #### Hoare logic for procedures: ```{P}C{Q}```
* #### Iris enables proof modularity/composition across functions


## Coq

### Programming Language


* #### Dependently-typed functional language, similar to OCaml/Haskell, first released in 1989
* #### Type system is based on the Calculus of Constructions, at the top of the lambda cube
````coq
Vector: Type := {v: list | len(v) = 2}
````
* #### Employs structural recursion for proof of termination
* #### Not just a Trusted Code Base (TCB) as Coq's implementation can be verified in Coq
* #### Programs can be transpiled ("extracted") to OCaml/Haskell/Scheme/C for execution


#### Example

In [1]:
Inductive list (X:Type) : Type :=
  | nil
  | cons (x : X) (l : list X).

Arguments nil {X}.
Arguments cons {X}.

Notation "x :: l" := (cons x l) (at level 60, right associativity).
Notation "[ ]" := nil.
Notation "[ x ; .. ; y ]" := (x :: .. (y :: nil) ..).


Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [129]:
Definition car {X: Type} (l: list X) : option X :=
  match l with
  | nil => None
  | x :: _ => Some x
  end.

Definition cdr {X: Type} (l: list X) : list X :=
  match l with
  | nil => nil
  | _ :: l' => l'
  end.

Fixpoint len {X: Type} (l: list X) : nat := 
  match l with
  | nil => 0
  | _ :: l' => (len l') + 1
  end.

Fixpoint concat {X: Type} (a b: list X) : list X :=
  match a with
  | nil => b
  | x :: l => x :: (concat l b)
  end.
  Notation "a ++ b" := (concat a b) (right associativity, at level 60).

Fixpoint reverse {X: Type} (l: list X) : list X :=
  match l with
  | nil => nil
  | x :: l => (reverse l) ++ [x]
  end.
  
Compute reverse ([1; 2] ++ [3; 4]).


= [4; 3; 2; 1]
     : list nat

### Interactive Proof Assistant


* #### A Trusted Theory Base (TTB), since Coq's semantics can't be proven in Coq due to Gödel
* #### Ultimately just a dependent type checker, via Curry-Howard correspondence
* #### Proofs are similar to handwritten two-column proofs - goals are transformed/reduced via tactics
* #### Proof structure: goal(s), local context, other proofs, tactics
* #### Downside: from the source alone, it is hard to see what a proof is doing


#### Example

In [26]:
Theorem cons_is_concat: 
  forall (x: nat) (l: list nat), 
    x :: l = [x] ++ l.
Proof.
  intros x l.
  simpl.
  auto.
Qed.


Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

## Iris

<blockquote style="width: 50%;">
<h4>
Iris is also, unfortunately, a beast to learn. It can be hard to tell where to start, hard to figure out what Iris even is, and hard to appreciate why Iris is such a good framework.
</h4>
</blockquote>

* #### A relatively new toolset (~2014)
* #### Has been used to prove a subset of the rust type system
* #### Language interfaces:
    * HeapLang (built-in toy language)
    * Go (goose)
    * Rust (rustbelt)
    * Scala, not that anyone cares


### Concurrent Separation Logic

* #### Enables reasononing about arbitrary imperative programs in Coq, including memory/pointers
* #### "Separation" involves partitioning memory into disjoint regions that can be reasoned independently
* #### An extension of Hoare logic `{P}C{Q}`
    * Function `C` executes with initial states satisfying a precondition `P`
    * If nothing goes wrong and `C` terminates, then the final state will satisfy postcondition `Q`
    * `C` may only access memory asserted by `P` or allocated by `C`
    * All memory addresses referred to by `P` are considered owned by a thread executing `C`
* #### Example
````ocaml
{x -> n} let v = !x in x <- v + 1 {x -> n + 1}
````

## Notes

### Proof to Code Ratio


<img src="loc.png" width="50%" style="margin: 0" />

### Coq Formatter

<img src="coqfmt.png" width="100%"/>


### QuickChick

* #### Randomized property-based testing framework, similar to Haskell's QuickCheck
* #### Enables proving that a test verifies a Coq property
* #### Automatically generates data from Coq's dependent type system


### Some Proven Systems

* #### WebAssembly specification
    * Soundness of the WASM type system
    * A verified WASM bytecode interpreter
    * Implemented using the Isabelle proof assistant
    
* #### CompCert optimizing compiler
    * Verifies a subset (CLight) of the semantics of C99
    * Initially designed to be a safe optimizing compiler for C for embedded systems
    * Implemented and verified in Coq
    * Employs a sequence of 12 intermediate languages during compilation


### Other Proof Assistants/Languages

* #### Isabelle
    * ML-based proof assistant
    * Archive of Formal Proofs (www.isa-afp.org) - a sort of package manager for proofs

* #### Lean
    * CoC-based Logic similar to Coq
    * Geared towards mathematics community
    * Transpiles to Javascript for the browser
    * Xena Project: rewrite all theorems/proofs in undergrad math in Lean

* #### F*
    * CoC-based proof assistant similar to Coq
    * Includes an SMT solver for some automated theorem proving
    * Steel project for concurrency reasoning, similar to Iris
    * Transpiles to OCaml/F#/C


### Weaker Results

* #### Facebook's Infer static analyzer uses Separation Logic (like Iris)
* #### They bought it from the Monoidics company in 2013
* #### Checks for null pointer problems, memory leaks, etc. in C/C++/Objective-C
* #### Implemented in OCaml


### Links

* https://www.chajed.io/papers/gojournal:osdi2021.pdf
* https://softwarefoundations.cis.upenn.edu/
* https://arxiv.org/abs/2105.12077
* https://cacm.acm.org/magazines/2019/2/234356-separation-logic/fulltext
* https://plv.csail.mit.edu/blog/iris-intro.html
* https://pjreddie.com/coq-tactics/
